In [ ]:
import sys
import os
import logging
sys.path.insert(1, os.path.join(sys.path[0], '../../src'))
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)

In [ ]:
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from darts.dataprocessing.transformers.scaler import Scaler
from sklearn.preprocessing import MinMaxScaler
from darts import concatenate

In [ ]:
from datasets import ShellDataset, WeatherEcadDataset
from visualization import plot_time_series_predictions

### Load Dataset

In [ ]:
dataset = ShellDataset(subset=3)
data = dataset.load_dataset()

# Transform splits
target_scaler = Scaler(MinMaxScaler())
train_series = target_scaler.fit_transform(data['train'])
test_series = target_scaler.transform(data['test'])

# Rebuild full series
train_test_series = concatenate([train_series, test_series])

In [ ]:
# Load Covariates
weather_dataset = WeatherEcadDataset()
past_covariates = weather_dataset.load_covariates()

# Scale Covariates
covariates_scaler = Scaler(MinMaxScaler())
past_covariates = covariates_scaler.fit_transform(past_covariates)

In [ ]:
print("Number of data points in train, test: ", len(data['train']),  len(data['test']))

### Set Experiment Parameters

In [ ]:
FORECAST_HORIZON = 1

In [ ]:
from darts.utils.timeseries_generation import holidays_timeseries

add_encoders = {
    'cyclic': {'future': ['month']},
    'datetime_attribute': {'future': ['dayofweek', 'weekday']},
    'position': {'past': ['relative'], 'future': ['relative']},
    'custom': {'past': [lambda idx: (idx.year - 1950) / 50]},
}

#### DeepAR

In [ ]:
from darts.models import BlockRNNModel
from darts.utils.likelihood_models import GaussianLikelihood
# from dart
def load_deeparmodel():
    return BlockRNNModel(
        model="LSTM",
        hidden_dim=150, 
        n_rnn_layers=3,
        dropout=0.2,
        batch_size=64,
        add_encoders=add_encoders,
        optimizer_kwargs={'lr': 1e-4}, 
        nr_epochs_val_period=1,
        # log_tensorboard=logdir,
        input_chunk_length=30,
        output_chunk_length=FORECAST_HORIZON,
        random_state=0,
        likelihood=GaussianLikelihood(),
        n_epochs=100,
        # loss_fn=torch.nn.MSELoss(),
        pl_trainer_kwargs={"callbacks": [EarlyStopping(monitor="val_loss", patience=10, min_delta=0.01, mode='min')], "log_every_n_steps": 1},
    )

In [ ]:
parameters = {
    "model": ["LSTM"],
    "batch_size": [32],
    "add_encoders": [add_encoders],
    "nr_epochs_val_period": [1],
    "input_chunk_length": [30],
    "output_chunk_length": [FORECAST_HORIZON],
    "likelihood": [GaussianLikelihood()],
    "hidden_dim": [16, 32, 64, 128],
    "n_rnn_layers": [1, 2, 3, 4],
    "dropout": [0.0, 0.1, 0.2, 0.3, 0.4],
    "optimizer_kwargs": [{"lr": 1e-3}, {"lr": 1e-4}, {"lr": 1e-5}],
    # "pl_trainer_kwargs": [{"callbacks": [EarlyStopping(monitor="val_loss", patience=10, min_delta=0.01, mode='min')], "log_every_n_steps": 1}],
}

In [ ]:
%%capture
train_series_single_train, train_series_single_valid = train_series.split_after(0.7)

model, params, score = BlockRNNModel.gridsearch(
    parameters,
    series=train_series_single_train,
    val_series=train_series_single_valid,
    past_covariates=past_covariates,
)

print(params)
